In [ ]:
import pandas as pd

#Reading data from csv
df=pd.read_csv("../data/raw_data.csv")
df.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0


In [ ]:
#Checking the basic info and dimensions of data sets
print(df.info())
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440833 entries, 0 to 440832
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CustomerID         440832 non-null  float64
 1   Age                440832 non-null  float64
 2   Gender             440832 non-null  object 
 3   Tenure             440832 non-null  float64
 4   Usage Frequency    440832 non-null  float64
 5   Support Calls      440832 non-null  float64
 6   Payment Delay      440832 non-null  float64
 7   Subscription Type  440832 non-null  object 
 8   Contract Length    440832 non-null  object 
 9   Total Spend        440832 non-null  float64
 10  Last Interaction   440832 non-null  float64
 11  Churn              440832 non-null  float64
dtypes: float64(9), object(3)
memory usage: 40.4+ MB
None
(440833, 12)


In [ ]:
#Coverting the column names for better accessing
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
print(df.columns)


Index(['customerid', 'age', 'gender', 'tenure', 'usage_frequency',
       'support_calls', 'payment_delay', 'subscription_type',
       'contract_length', 'total_spend', 'last_interaction', 'churn'],
      dtype='object')


In [ ]:
#Handling missing values
df['customerid']=df['customerid'].fillna(df['customerid'].median())
df['tenure']=df['tenure'].fillna(df['tenure'].median())


In [ ]:
#Fix the data type of churn
df['churn'] = df['churn'].astype('Int64')


In [ ]:
#Change the categorical columns into numbers
df = pd.get_dummies(df,columns=['gender', 'subscription_type', 'contract_length'],
    drop_first=True)


In [ ]:
#Creating new feature columns for better understanding of customer behaviour
# Customer value: How valuable the customer is
df['avg_spend_per_tenure'] = df['total_spend'] / (df['tenure'] + 1)

# Support intensity: How often the customer needs support
df['support_calls_per_tenure'] = df['support_calls'] / (df['tenure'] + 1)

# Payment behavior: Payment problem or not
df['delayed_payment_flag'] = (df['payment_delay'] > 0).astype('Int64')

# Engagement score:  How active the customer is
df['engagement_score'] = (df['usage_frequency'] /(df['support_calls'] + 1))


In [ ]:
#Removing the outliers(Extreme data)
Q1 = df['total_spend'].quantile(0.25)
Q3 = df['total_spend'].quantile(0.75)
IQR = Q3 - Q1

df = df[(df['total_spend'] >= Q1 - 1.5 * IQR) &(df['total_spend'] <= Q3 + 1.5 * IQR)]


In [ ]:
#Checking the correlation
corr = df.corr(numeric_only=True)
corr['churn'].sort_values(ascending=False)


churn                         1.000000
support_calls                 0.574267
contract_length_Monthly       0.433553
payment_delay                 0.312129
support_calls_per_tenure      0.265607
age                           0.218394
last_interaction              0.149616
delayed_payment_flag          0.040468
subscription_type_Standard   -0.009247
subscription_type_Premium    -0.011072
usage_frequency              -0.046101
tenure                       -0.051919
avg_spend_per_tenure         -0.069838
gender_Male                  -0.175395
contract_length_Quarterly    -0.176244
engagement_score             -0.342566
total_spend                  -0.429355
customerid                   -0.839365
Name: churn, dtype: float64

In [ ]:
#Dropping the columns which are not useful
df.drop(columns=['customerid'], inplace=True)


In [ ]:
#Saving the clean data
df.to_csv("../data/processed_data.csv", index=False)
